In [1]:
from fastai.text.all import *
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
%config Completer.use_jedi = False

In [2]:
class CharacterTokenizer():
    
    def __init__(self, split_char=' ', **kwargs): 
        self.split_char=split_char
        
    def __call__(self, items):
        
        # List where I temporarly store the tokens ['xxbos', 'h', 'e', 'l', 'l', 'o', 'xxeos'] as 
        # they are being parsed.
        final_list = []
        
        # We don't want to mess with the special fastai tokens
        special_chars = ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj']
        
        # Break up string into words, if word in special_chars dont touch it. Otherwise break up each
        # word into each character.
        for words in items:
            for word in words.split():
                if word not in special_chars:
                    for char in word:
                        final_list.append([char])
                else:
                    final_list.append([word])

        # Return a generator?? I'm not sure why we need to do this...        
        return (x for x in final_list)


SyntaxError: invalid syntax (<ipython-input-2-7f65592b8f10>, line 26)

In [ ]:
# Example of a validation dataset. In this case it is a list of python dictionaries.
# You should be able to adapt this to your usecase if you can read in a file using Pandas.
valid_jsons = [
    {"from_txt":"Hello how are you?","to_txt":"I am doing fine."},
    {"from_txt":"Is it going to rain today?","to_txt":"Let me pull up the weather."},
    {"from_txt":"How do fastai DataBlocks work?","to_txt":"Not sure, I'm still learning."}
]

# Read the list of dicts into a Data Frame
df_valid = pd.DataFrame(valid_jsons)

# Add a is_valid colum. The ColSplitter() function below expects this column.
df_valid['is_valid'] = True

# View our work
df_valid.head()

In [ ]:
# Example of a test dataset. In this case it is a list of python dictionaries.
# You should be able to adapt this to your usecase if you can read in a file using Pandas.
test_jsons = [
    {"from_txt":"Hello, where is the closest McDonald's?","to_txt":"Let me find you that on Google Maps."},
    {"from_txt":"Is it going to snow today?","to_txt":"Let me pull up the weather."},
    {"from_txt":"How much coffee is safe to drink?","to_txt":"As much as you need to learn the Fastai Library."}
]

# Read the list of dicts into a Data Frame
df_train = pd.DataFrame(test_jsons)


# Add a is_valid colum
df_train['is_valid'] = False

# View our work
df_train.head()

In [ ]:
# Stack the two dataframes into one.
df = pd.concat([df_train,df_valid], ignore_index=True)
df.head()

In [ ]:
logs = DataBlock(
    
    # blocks specify what type of data we are going to be loading.
    # In this case both are text files contained in the same df
    #blocks=(TextBlock.from_df('from_txt',is_lm=False),TextBlock.from_df('to_txt',is_lm=False)),
    
    # You can specify a tokenizer by passing in a tok variable. Comment the line above and ucomment the onces below.
    #blocks=(
    #    TextBlock.from_df('from_txt', is_lm=False, tok=SubwordTokenizer(vocab_sz=200)),
    #    TextBlock.from_df('to_txt'  , is_lm=False, tok=SubwordTokenizer(vocab_sz=200))),
    
    blocks=(
        TextBlock.from_df('from_txt', is_lm=False, tok=CharacterTokenizer()),
        TextBlock.from_df('to_txt'  , is_lm=False, tok=CharacterTokenizer())),
    
    # The TestBlock tokenization process puts tokenized inputs into a column called text. 
    # The ColReader for get_x will always reference text, even if the original text inputs 
    # were in a column with another name in the dataframe.
    get_x=ColReader('text'),
    get_y=ColReader('text'),
    
    # The dataframe needs to have a is_valid column for this to work.
    splitter=ColSplitter()

)

In [ ]:
dls = logs.dataloaders(df)
#dls.show_batch(max_n=2)